## FDW Crop Production Data Profiling - Somalia

In [1]:
import os, sys, glob, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf, save_npz, load_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_GrainTypeAgg, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools_graphic import PlotBarProduction, PlotLinePAY, PlotHeatCropSystem, PlotHeatSeasonData
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('./data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:32639' # Somalia
fn_shapes = sorted(glob.glob('./data/shapefile/fewsnet/SO_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
auth = tuple(json.loads(open('token.json', "r").read()))
parameters = {
    'format': 'json',
    'country': 'Somalia',
    'product': 'R011',
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy()
# ----------------------------------------------- #

# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# 2. Manual setting
# a) Add names to reporting units 
reporting_unit = df.loc[df['fnid'].map(lambda x: x[6] != 'A'), 'fnid'].unique()
df.loc[df.fnid.isin(reporting_unit),'name'] = df.loc[df.fnid.isin(reporting_unit), 'locality_name'].map(lambda x: x.split(' ')[0])
# # b) Delay one year of Deyr season data 
# # This is from an ongoing discussion. Currently, we confirmed that the Deyr season data is year off.
# # For example, the Deyr season data when season_date == 2019-02-01 corresponds to the 2020-02-01 data.
# idx_deyr = df['season_name'] == 'Deyr'
# delayed = pd.to_datetime(df.loc[idx_deyr, 'season_date']) + pd.offsets.DateOffset(years=1)
# df.loc[idx_deyr, 'season_date'] = delayed.dt.strftime('%Y-%m-%d')
# c) Control population groups
# Data of Awdal (SO1990A111) prior to 2017 has a population group (Borama (SO1990A21109) and Baki (SO1990A21102))
# This is not duplicated data, so we will link the data with an exception for one duplicated data point:
df = df[df['id'] != 27061759]
# ----------------------------------------------- #

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df)
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([SO_Admin1_1990, SO_Admin2_1990], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   |
| 1990	 | SO1990A1 | 18	| SO1990A2	| 74	|
----------------------------------------------- #


In [54]:
# temp = df[
#     (df['season_year'] == 'Gu 2021') &
#     (df['product'] == 'Maize (Corn)') &
# #     (df['product'] == 'Sorghum') &
# #     (df['crop_production_system'] == 'agro_pastoral') &
#     (df['crop_production_system'] == 'riverine') &
#     # (df['crop_production_system'] == 'riverine') &
#     (df['indicator'] == 'Quantity Produced')
# ].reset_index(drop=True)
# temp[['admin_1','admin_2','crop_production_system','season_year','value']]

,admin_1,admin_2,crop_production_system,season_year,value
0,Hiiraan,Beledweyn,riverine,Gu 2021,40.0
1,Hiiraan,Bulo-Burte,riverine,Gu 2021,200.0
2,Hiiraan,Jalalaqsi,riverine,Gu 2021,110.0
3,Middle Shabelle,Jowhar,riverine,Gu 2021,3000.0
4,Middle Shabelle,Balcad,riverine,Gu 2021,3500.0
5,Lower Shabelle,Afgooye,riverine,Gu 2021,2160.0
6,Lower Shabelle,Kurtunwaarey,riverine,Gu 2021,2500.0
7,Lower Shabelle,Qoryooley,riverine,Gu 2021,1540.0
8,Lower Shabelle,Sablaale,riverine,Gu 2021,1440.0
9,Gedo,Garbahaarey,riverine,Gu 2021,360.0


In [42]:
temp['value'].sum()

57142.2

In [19]:
df[['start_date', 'period_date', 'season_name','season_type', 'season_date', 'season_year']]

,start_date,period_date,season_name,season_type,season_date,season_year
0,2020-08-01,2020-08-31,Gu,Harvest,2020-07-01,Gu 2020
1,2020-08-01,2020-08-31,Gu,Harvest,2020-07-01,Gu 2020
2,2019-08-01,2019-08-31,Gu,Harvest,2019-07-01,Gu 2019
3,2019-08-01,2019-08-31,Gu,Harvest,2019-07-01,Gu 2019
4,2019-02-01,2019-02-28,Deyr,Harvest,2018-10-01,Deyr 2018
...,...,...,...,...,...,...
17039,1998-02-01,1998-02-28,Deyr,Harvest,1997-10-01,Deyr 1997
17040,1997-08-01,1997-08-31,Gu,Harvest,1997-07-01,Gu 1997
17041,1997-02-01,1997-02-28,Deyr,Harvest,1996-10-01,Deyr 1996
17042,1997-02-01,1997-02-28,Deyr,Harvest,1996-10-01,Deyr 1996


In [9]:
tmp = df_origin.copy()
tmp[['Somalia']]

,country,source_organization,source_document,geographic_unit_full_name,geographic_unit_name,fnid,admin_0,admin_1,admin_2,admin_3,...,dataseries_specialization_type,data_usage_policy,created,modified,status_changed,collection_status,collection_status_changed,collection_schedule,collection_date,publication_name
0,Somalia,"FSNAU, Somalia","FSNAU, Somalia","Awdal, Somalia",Awdal,SO1990A111,Somalia,Awdal,None,None,...,/SurveyPopulationIndicator/CropProductionIndic...,Public,2021-11-04T18:50:52,2021-11-04T18:50:52,2021-11-04T18:51:08,Published,2021-11-04T18:57:09,Ad Hoc,2020-08-31,
1,Somalia,"FSNAU, Somalia","FSNAU, Somalia","Awdal, Somalia",Awdal,SO1990A111,Somalia,Awdal,None,None,...,/SurveyPopulationIndicator/CropProductionIndic...,Public,2021-11-04T18:50:52,2021-11-04T18:50:52,2021-11-04T18:51:08,Published,2021-11-04T18:57:09,Ad Hoc,2020-08-31,
2,Somalia,"FSNAU, Somalia","FSNAU, Somalia","Awdal, Somalia",Awdal,SO1990A111,Somalia,Awdal,None,None,...,/SurveyPopulationIndicator/CropProductionIndic...,Public,2021-11-04T18:50:45,2021-11-04T18:50:45,2021-11-04T18:51:08,Published,2021-11-04T18:57:09,Ad Hoc,2019-08-31,
3,Somalia,"FSNAU, Somalia","FSNAU, Somalia","Awdal, Somalia",Awdal,SO1990A111,Somalia,Awdal,None,None,...,/SurveyPopulationIndicator/CropProductionIndic...,Public,2021-11-04T18:50:45,2021-11-04T18:50:45,2021-11-04T18:51:08,Published,2021-11-04T18:57:09,Ad Hoc,2019-08-31,
4,Somalia,"FSNAU, Somalia","FSNAU, Somalia","Awdal, Somalia",Awdal,SO1990A111,Somalia,Awdal,None,None,...,/SurveyPopulationIndicator/CropProductionIndic...,Public,2019-10-11T20:07:19,2019-10-11T20:07:30,2019-10-11T20:07:30,Published,2021-08-31T15:25:30,Ad Hoc,2019-02-28,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17039,Somalia,"FSNAU, Somalia","FSNAU, Somalia","Hagar, Lower Juba, Somalia",Hagar,SO1990R22806,Somalia,Lower Juba,Afmadow,None,...,/SurveyPopulationIndicator/CropProductionIndic...,Public,2018-04-18T10:00:08,2018-04-18T10:04:10,2018-04-18T10:04:10,Published,2019-08-09T14:06:27,Ad Hoc,1998-02-28,"FSNAU Spreadsheet, 2017, Somalia"
17040,Somalia,"FSNAU, Somalia","FSNAU, Somalia","Hagar, Lower Juba, Somalia",Hagar,SO1990R22806,Somalia,Lower Juba,Afmadow,None,...,/SurveyPopulationIndicator/CropProductionIndic...,Public,2018-04-18T10:00:06,2018-04-18T10:04:10,2018-04-18T10:04:10,Published,2019-08-09T14:06:27,Ad Hoc,1997-08-31,"FSNAU Spreadsheet, 2017, Somalia"
17041,Somalia,"FSNAU, Somalia","FSNAU, Somalia","Hagar, Lower Juba, Somalia",Hagar,SO1990R22806,Somalia,Lower Juba,Afmadow,None,...,/SurveyPopulationIndicator/CropProductionIndic...,Public,2018-04-18T10:00:03,2018-04-18T10:04:10,2018-04-18T10:04:10,Published,2019-08-09T14:06:27,Ad Hoc,1997-02-28,"FSNAU Spreadsheet, 2017, Somalia"
17042,Somalia,"FSNAU, Somalia","FSNAU, Somalia","Hagar, Lower Juba, Somalia",Hagar,SO1990R22806,Somalia,Lower Juba,Afmadow,None,...,/SurveyPopulationIndicator/CropProductionIndic...,Public,2018-04-18T10:00:03,2018-04-18T10:04:10,2018-04-18T10:04:10,Published,2019-08-09T14:06:27,Ad Hoc,1997-02-28,"FSNAU Spreadsheet, 2017, Somalia"


In [11]:
tmp[
    
]

,country,fnid,admin_1,admin_2,name,population_group,crop_production_system,start_date,period_date,season_name,season_type,season_date,season_year,year,product,indicator,value,locality_name
0,Somalia,SO1990A111,Awdal,None,Awdal,none,agro_pastoral,2020-08-01,2020-08-31,Gu,Harvest,2020-07-01,Gu 2020,2020,Sorghum,Area Harvested,6200.000000,"Awdal, Somalia (From 01/01/1990): Agropastoral"
1,Somalia,SO1990A111,Awdal,None,Awdal,none,agro_pastoral,2020-08-01,2020-08-31,Gu,Harvest,2020-07-01,Gu 2020,2020,Maize (Corn),Area Harvested,770.000000,"Awdal, Somalia (From 01/01/1990): Agropastoral"
2,Somalia,SO1990A111,Awdal,None,Awdal,none,agro_pastoral,2019-08-01,2019-08-31,Gu,Harvest,2019-07-01,Gu 2019,2019,Maize (Corn),Area Harvested,200.000000,"Awdal, Somalia (From 01/01/1990): Agropastoral"
3,Somalia,SO1990A111,Awdal,None,Awdal,none,agro_pastoral,2019-08-01,2019-08-31,Gu,Harvest,2019-07-01,Gu 2019,2019,Sorghum,Area Harvested,7500.000000,"Awdal, Somalia (From 01/01/1990): Agropastoral"
4,Somalia,SO1990A111,Awdal,None,Awdal,none,agro_pastoral,2019-02-01,2019-02-28,Deyr,Harvest,2018-10-01,Deyr 2018,2018,Sorghum,Area Harvested,8500.000000,"Awdal, Somalia (From 01/01/1990): Agropastoral"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12004,Somalia,SO1990R22806,Lower Juba,Afmadow,"Hagar,",none,none,1999-02-01,1999-02-28,Deyr,Harvest,1998-10-01,Deyr 1998,1998,Sorghum,Yield,0.400000,"Hagar, Lower Juba, Somalia (From 01/01/1990)"
12005,Somalia,SO1990R22806,Lower Juba,Afmadow,"Hagar,",none,none,1998-08-01,1998-08-31,Gu,Harvest,1998-07-01,Gu 1998,1998,Maize (Corn),Yield,0.799762,"Hagar, Lower Juba, Somalia (From 01/01/1990)"
12006,Somalia,SO1990R22806,Lower Juba,Afmadow,"Hagar,",none,none,1998-02-01,1998-02-28,Deyr,Harvest,1997-10-01,Deyr 1997,1997,Maize (Corn),Yield,0.500000,"Hagar, Lower Juba, Somalia (From 01/01/1990)"
12007,Somalia,SO1990R22806,Lower Juba,Afmadow,"Hagar,",none,none,1997-02-01,1997-02-28,Deyr,Harvest,1996-10-01,Deyr 1996,1996,Sorghum,Yield,0.700000,"Hagar, Lower Juba, Somalia (From 01/01/1990)"


In [7]:
df.columns

Index(['country', 'fnid', 'admin_1', 'admin_2', 'name', 'population_group',
       'crop_production_system', 'start_date', 'period_date', 'season_name',
       'season_type', 'season_date', 'season_year', 'year', 'product',
       'indicator', 'value', 'locality_name'],
      dtype='object')

In [5]:
df_origin[['start_date', 'period_date', 'season_name','season_type', 'season_date', 'season_year',]].drop_duplicates()

,start_date,period_date,season_name,season_type,season_date,season_year
0,2020-08-01,2020-08-31,Gu,Harvest,2020-07-01,Gu 2020
2,2019-08-01,2019-08-31,Gu,Harvest,2019-07-01,Gu 2019
4,2019-02-01,2019-02-28,Deyr,Harvest,2018-10-01,Deyr 2018
5,2018-08-01,2018-08-31,Gu,Harvest,2018-07-01,Gu 2018
6,2018-02-01,2018-02-28,Deyr,Harvest,2017-10-01,Deyr 2017
...,...,...,...,...,...,...
13766,2009-10-01,2009-10-31,Gu off-season,Harvest,2009-09-01,Gu off-season 2009
13769,2009-03-01,2009-03-31,Deyr off-season,Harvest,2009-03-01,Deyr off-season 2009
13774,2008-03-01,2008-03-31,Deyr off-season,Harvest,2008-03-01,Deyr off-season 2008
13785,2005-10-01,2005-10-31,Gu off-season,Harvest,2005-09-01,Gu off-season 2005


In [4]:
df[['start_date', 'period_date', 'season_name','season_type', 'season_date', 'season_year',]].drop_duplicates()

,start_date,period_date,season_name,season_type,season_date,season_year
0,2020-08-01,2020-08-31,Gu,Harvest,2020-07-01,Gu 2020
2,2019-08-01,2019-08-31,Gu,Harvest,2019-07-01,Gu 2019
4,2019-02-01,2019-02-28,Deyr,Harvest,2018-10-01,Deyr 2018
5,2018-08-01,2018-08-31,Gu,Harvest,2018-07-01,Gu 2018
6,2018-02-01,2018-02-28,Deyr,Harvest,2017-10-01,Deyr 2017
...,...,...,...,...,...,...
9714,2009-10-01,2009-10-31,Gu off-season,Harvest,2009-09-01,Gu off-season 2009
9717,2009-03-01,2009-03-31,Deyr off-season,Harvest,2009-03-01,Deyr off-season 2009
9722,2008-03-01,2008-03-31,Deyr off-season,Harvest,2008-03-01,Deyr off-season 2008
9733,2005-10-01,2005-10-31,Gu off-season,Harvest,2005-09-01,Gu off-season 2005


- Somalia crop seasonal calendar </br>

![FEWS NET](https://fews.net/sites/default/files/styles/large/public/seasonal-calendar-somalia.png?itok=6LQ1arWc)

- FDW data consists of `SO1990A1` and `SO1990A2`.

| Year | Admin-1 | # units | Admin-2  | # units |
| :--- | :----:  | :----:  | :----:   | ---:    |
| 1990 | SO1990A1| 18      | **`SO1990A2`** | 74      |

- Comparison between admin boundaries.

![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_admin_shapes.png?raw=true)

<!-- - Most of reliable data are admin-2 level, and FDW has few data for admin-1 level. Therefore, we aggregate/merge admin-2 level data to represent admin-1 level. -->
- Afmadow (SO1990A22802) has two crop reporting units: Afmadow (SO1990R22805) and Afmadow (Hagar) (SO1990R22806). This is confirmed by FEWS NET crop reporting unit (./data/admin_boundaries/SO_Crop_Units_1990.shp). Since SO1990A22802 has only one off-season record, we aggregate SO1990R22805 and SO1990R22806 to replace SO1990A22802.

| District |Crop reporting units|
| :---:|:---:|
|Afmadow (SO1990A22802) | Afmadow (SO1990R22805) and  Hagar (SO1990R22806)|

- **`SO1990A2`** is used to represent the current admin-level 2 crop data.
- Somalia has four crop seasons: `Deyr`, `Gu`, `Gu off-season`, `Deyr off-season`.
- Somalia has three crop production systems: `agro_pastoral`, `riverine`, `None`.
- Somalia has no population group(s).

In Admin-2 level data, we found unrealistic records. So, we manually remove those records:
- 26 districts have the exactly same `1 mt/ha` yield values in `Deyr` season of `2001`.
- Three districts have unrealistic yield values in `Gu` season of `2005`: Bu'aale (11.65 mt/ha), Jilib (2.27 mt/ha), and Saakow (20.8 mt/ha).

In [3]:
# Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = SO_Admin2_1990.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
df['season_name'] = df['season_name'].replace({'Gu off-season':'Gu-off', 'Deyr off-season':'Deyr-off'})
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #

In [4]:
# Link admin boundaries ------------------------- #
link, over = FDW_PD_CreateAdminLink(SO_Admin2_1990, SO_Admin2_1990, 'ADMIN2', 'ADMIN2', area, epsg)
# Crop specific ratios
link_ratio = FDW_PD_RatioAdminLink(link, area, over, mdx_pss)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Manual Editing
link_ratio['SO1990A22802'][['SO1990R22805', 'SO1990R22806']] = 1.0
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=False)
# ----------------------------------------------- #

# Aggregate grain data by grain type ------------ #
[area_new, prod_new, area_all, prod_all] = FDW_PD_GrainTypeAgg([area_new, prod_new, area_all, prod_all], product_category)
# ----------------------------------------------- #

# Manual correction ----------------------------- #
crop_new = prod_new/area_new
# - Remove data before 1995
area_new = area_new.loc[1995:]
prod_new = prod_new.loc[1995:]
# - Unrealistic values in 2005 in Gu season
target = ['SO1990A22701','SO1990A22702','SO1990A22703']
area_new.loc[2005,pd.IndexSlice[target,:,'Maize','Gu',:]] = np.nan
prod_new.loc[2005,pd.IndexSlice[target,:,'Maize','Gu',:]] = np.nan
# # - Districts have the same 1 mt/ha yield records in 2001 in Deyr season
# temp = crop_new.loc[2001, pd.IndexSlice[:,:,'Maize','Deyr']]
# target = temp.index.get_level_values(0)[temp == 1]
# area_new.loc[2001,pd.IndexSlice[target,:,'Maize','Deyr',:]] = np.nan
# prod_new.loc[2001,pd.IndexSlice[target,:,'Maize','Deyr',:]] = np.nan
# # - Unrealistic values in 1995 in Dery season
# target = ['SO1990A22605']
# area_new.loc[1995,pd.IndexSlice[target,:,'Maize','Deyr',:]] = np.nan
# prod_new.loc[1995,pd.IndexSlice[target,:,'Maize','Deyr',:]] = np.nan
# ----------------------------------------------- #

# Complete long format DataFrame
df_area = area_new.T.stack().reset_index().rename({0:'value'},axis=1)
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().reset_index().rename({0:'value'},axis=1)
df_prod['indicator'] = 'production'
df_yield = (prod_new/area_new).T.stack().reset_index().rename({0:'value'},axis=1)
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Insert a country name
stack['country'] = 'Somalia'
stack = stack[['fnid','country','name','product','year','season_name','season_date','indicator','value']]
stack = stack.reset_index(drop=True)
# Change season_date to harvest_end
stack.rename(columns={'season_date':'harvest_end'},inplace=True)
stack['harvest_end'] = stack['harvest_end'].replace({
    '07-01':'07-01', # Gu
    '02-01':'02-01', # Deyr
    '09-01':'09-01', # Gu off-season
    '03-01':'03-01', # Deyr off-season
})

# Save data
save_hdf('./data/crop/adm_crop_production_raw_SO.hdf', df)
save_hdf('./data/crop/adm_crop_production_SO.hdf', stack)
save_npz('./data/crop/adm_crop_production_SO_ratio.npz', link_ratio)

CBR is considered for 'SO1990A21604' as no record found in: ['SO1990A21604']
CBR is considered for 'SO1990A21202' as no record found in: ['SO1990A21202']
CBR is considered for 'SO1990A21102' as no record found in: ['SO1990A21102']
CBR is considered for 'SO1990A21703' as no record found in: ['SO1990A21703']
CBR is considered for 'SO1990A21802' as no record found in: ['SO1990A21802']
CBR is considered for 'SO1990A21103' as no record found in: ['SO1990A21103']
CBR is considered for 'SO1990A21603' as no record found in: ['SO1990A21603']
CBR is considered for 'SO1990A21605' as no record found in: ['SO1990A21605']
CBR is considered for 'SO1990A21104' as no record found in: ['SO1990A21104']
CBR is considered for 'SO1990A21601' as no record found in: ['SO1990A21601']
CBR is considered for 'SO1990A21503' as no record found in: ['SO1990A21503']
CBR is considered for 'SO1990A21502' as no record found in: ['SO1990A21502']
CBR is considered for 'SO1990A21606' as no record found in: ['SO1990A21606']

## Visualization of production data

In [5]:
# Bar chart of national grain production
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
year = [df['year'].min(), df['year'].max()]
product_order = ['Maize','Sorghum','Rice']
for season_name in ['Gu','Deyr']:
    footnote = 'National grain production in %s - %s' % (country_name, season_name)
    fn_save = './figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote, fn_save)
    # fig.show()

./figures/SO_bar_natgrainprod_Gu.png is saved.
./figures/SO_bar_natgrainprod_Deyr.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_bar_natgrainprod_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_bar_natgrainprod_Deyr.png?raw=true)

In [6]:
# Lineplot of Production-Area-Yield (PAY) time-series
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Maize','Gu'],
    ['Maize','Deyr'],
    ['Sorghum','Gu'],
    ['Sorghum','Deyr'],    
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield (PAY) time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = './figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    # fig.show()

./figures/SO_line_pay_Maize_Gu.png is saved.
./figures/SO_line_pay_Maize_Deyr.png is saved.
./figures/SO_line_pay_Sorghum_Gu.png is saved.
./figures/SO_line_pay_Sorghum_Deyr.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_line_pay_Maize_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_line_pay_Maize_Deyr.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_line_pay_Sorghum_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_line_pay_Sorghum_Deyr.png?raw=true)

In [7]:
# Heatmap of seasonal data availability
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('./data/crop/adm_crop_production_raw_%s.hdf' % country_iso)
code = {'Gu':1,'Deyr':10}
comb = {1:1,10:2,11:3,12:3,20:3,21:3,22:3}
comb_name = {1:'Gu',2:'Deyr',3:'Gu + Deyr'}
for product_name in ['Maize (Corn)','Sorghum']:
    data = df[(df['product'] == product_name) & (df['season_name'].isin(code.keys()))]
    footnote = 'Seasonal data availability in %s - %s (uncorrected)' % (country_name, product_name)
    fn_save = './figures/%s_heat_seasondata_%s.png' % (country_iso, product_name)
    fig = PlotHeatSeasonData(data, code, comb, comb_name, footnote, fn_save)
    # fig.show()

./figures/SO_heat_seasondata_Maize (Corn).png is saved.
./figures/SO_heat_seasondata_Sorghum.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_heat_seasondata_Maize%20(Corn).png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_heat_seasondata_Sorghum.png?raw=true)

In [8]:
# Heatmap of crop production system(s) per crop type
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('./data/crop/adm_crop_production_raw_%s.hdf' % country_iso)
product_season = [
    ['Maize (Corn)', 'Gu'],
    ['Maize (Corn)', 'Deyr'],
]
code = {'none':1,'riverine':10,'agro_pastoral':100}
comb = {1:1,10:2,11:3,100:4,101:5,110:6,111:7}
comb_name = {1:'None',2:'Riverine',3:'Riverine + None',4:'Agro_pastoral',5:'Agro_pastoral + None',6:'Agro_pastoral + Riverine',7:'All'}
for product_name, season_name in product_season:
    data = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    footnote = 'Reported crop production system(s) in %s - %s - %s (uncorrected)' % ('Somalia', product_name, season_name)
    fn_save = './figures/%s_heat_cropsystem_%s_%s.png' % (country_iso, product_name, season_name)
    fig = PlotHeatCropSystem(data, code, comb, comb_name, footnote, fn_save)
    # fig.show()

./figures/SO_heat_cropsystem_Maize (Corn)_Gu.png is saved.
./figures/SO_heat_cropsystem_Maize (Corn)_Deyr.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_heat_cropsystem_Maize%20(Corn)_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_heat_seasondata_SorghumSO_heat_cropsystem_Maize%20(Corn)_Deyr.png?raw=true)

### Irrigated (or riverine) districts

In [9]:
from tools_graphic import PlotMapCropSystem
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('./data/crop/adm_crop_production_raw_%s.hdf' % country_iso)
product_season = [
    ['Maize (Corn)', 'Gu'],
    ['Maize (Corn)', 'Deyr'],
    ['Sorghum', 'Gu'],
    ['Sorghum', 'Deyr']
]
for product_name, season_name in product_season:
    sub = df[(df['product'] == product_name)&(df['season_name'] == season_name)]
    footnote = 'Crop production system(s) in %s - %s - %s' % ('Somalia', product_name, season_name)
    fn_save = './figures/%s_map_cropsystem_%s_%s.png' % (country_iso, product_name, season_name)
    fig = PlotMapCropSystem(sub, footnote, fn_save)
    # fig.show()

./figures/SO_map_cropsystem_Maize (Corn)_Gu.png is saved.
./figures/SO_map_cropsystem_Maize (Corn)_Deyr.png is saved.
./figures/SO_map_cropsystem_Sorghum_Gu.png is saved.
./figures/SO_map_cropsystem_Sorghum_Deyr.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_map_cropsystem_Maize%20(Corn)_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_map_cropsystem_Maize%20(Corn)_Deyr.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_map_cropsystem_Sorghum_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_map_cropsystem_Sorghum_Deyr.png?raw=true)

In [10]:
# Calibrated PAY time-series per FNID
from tools_graphic import PlotLineCropTS
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
link_ratio = load_npz('./data/crop/adm_crop_production_%s_ratio.npz' % country_iso)
year_all = np.arange(df['year'].min(), df['year'].max()+1)
prod_season = [
    ['Maize', 'Gu'],
    ['Sorghum', 'Gu'],
    ['Maize', 'Deyr'],
    ['Sorghum', 'Deyr']
]
for product_name, season_name in prod_season:
    sub = df[
        (df['product'] == product_name) &
        (df['season_name'] == season_name)
    ]
    for fnid in sub['fnid'].unique():
        sub_fps = sub[sub['fnid'] == fnid]
        fn_save = './figures/crop_calibrated/%s_%s_%s_%s.png' % (country_iso, product_name, season_name, fnid)
        # fig = PlotLineCropTS(sub_fps, fnid, product_name, season_name, link_ratio, year_all, fn_save)